In [ ]:
import pandas as pd

In [ ]:
books = pd.read_csv('books_cleaned.csv')

In [ ]:
books['categories'].value_counts().reset_index()

In [ ]:
books['categories'].value_counts().reset_index().query('count > 50')

In [ ]:
category_mapping = {
    'Fiction' : "Fiction",
    'Juvenile Fiction': "Children's Fiction",
    'Biography & Autobiography': "Nonfiction",
    'History': "Nonfiction",
    'Literary Criticism': "Nonfiction",
    'Philosophy': "Nonfiction",
    'Religion': "Nonfiction",
    'Comics & Graphic Novels': "Fiction",
    'Drama': "Fiction",
    'Juvenile Nonfiction': "Children's Nonfiction",
    'Science': "Nonfiction",
    'Poetry': "Fiction"
    }

books['simple_categories'] = books['categories'].map(category_mapping)

In [ ]:
books[~(books['simple_categories'].isna())].shape

In [ ]:
from transformers import pipeline
fiction_categories = [
    "Fiction",
    "Nonfiction"]
pipe = pipeline("zero-shot-classification",
                model="facebook/bart-large-mnli",
                device="cuda")

In [ ]:
sequence = books.loc[books['simple_categories']=="Fiction", 'description'].reset_index(drop=True)[0]

In [ ]:
pipe(sequence,fiction_categories)

In [ ]:
import numpy as np
max_index = np.argmax(pipe(sequence,fiction_categories)['scores'])
max_label = pipe(sequence,fiction_categories)['labels'][max_index]

max_label

In [ ]:
def generate_predictions(sequence, categories):
    results = pipe(sequence, categories)
    max_index = np.argmax(results['scores'])
    max_label = results['labels'][max_index]
    return max_label

In [ ]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 300)):
    sequence = books.loc[books['simple_categories']=="Fiction", 'description'].reset_index(drop=True)[i]
    predicted_cats.append(generate_predictions(sequence, fiction_categories))
    actual_cats.append("Fiction")

In [ ]:
for i in tqdm(range(0, 300)):
    sequence = books.loc[books['simple_categories']=="Nonfiction", 'description'].reset_index(drop=True)[i]
    predicted_cats.append(generate_predictions(sequence, fiction_categories))
    actual_cats.append("Nonfiction")

In [ ]:
predictions_df = pd.DataFrame({
    'actual': actual_cats,
    'predicted': predicted_cats
})

In [ ]:
predictions_df['correct_prediction'] = (
    np.where(predictions_df['actual'] == predictions_df['predicted'], 1, 0)
)

In [ ]:
predictions_df['correct_prediction'].sum() / predictions_df.shape[0]

In [ ]:
isbns = []
predicted_cats = []
missing_cats = books.loc[books['simple_categories'].isna(), ['isbn13', "description"]].reset_index(drop=True)

In [ ]:
for i in tqdm(range(0, missing_cats.shape[0])):
    sequence = missing_cats['description'][i]
    pred = generate_predictions(sequence, fiction_categories)
    predicted_cats.append(pred)
    isbns.append(missing_cats['isbn13'][i])


In [ ]:
missing_predictions_df = pd.DataFrame({
    'isbn13': isbns,
    'predicted': predicted_cats
})

In [ ]:
missing_predictions_df

In [ ]:
books.columns

In [ ]:
asdf

In [ ]:
books = pd.merge(books, missing_predictions_df, on='isbn13', how='left')
books['simple_categories'] = np.where(books['simple_categories'].isna(),books['predicted'], books['simple_categories'])
books = books.drop(columns=['predicted'])


In [ ]:
books.head(2)

In [ ]:
books[books["categories"].str.lower().isin([
    "romance",
    "science fiction",
    "scifi",
    "fantasy",
    "horror",
    "mystery",
    "thriller",
    "comedy",
    "crime",
    "historical"
])]

In [ ]:
books.to_csv("books_with_categories.csv", index=False)